In [1]:
import pandas as pd
import glob
import os

folder_path = '.' 
csv_files = glob.glob(os.path.join(folder_path, '*.csv'))

delay_threshold = 120  # Delay more than 2hours

for file in csv_files:
    df = pd.read_csv(file, encoding='ISO-8859-1')
    
    # Convert FlightDate to datetime
    df['FlightDate'] = pd.to_datetime(df['FlightDate'], errors='coerce')
    
    # Convert time columns to datetime format
    df['CRSDepTime_timezone'] = pd.to_datetime(df['CRSDepTime_timezone'], errors='coerce')
    df['DepTime_timezone'] = pd.to_datetime(df['DepTime_timezone'], errors='coerce')
    df['CRSArrTime_timezone'] = pd.to_datetime(df['CRSArrTime_timezone'], errors='coerce')
    df['ArrTime_timezone'] = pd.to_datetime(df['ArrTime_timezone'], errors='coerce')
    
    year = df['FlightDate'].dt.year.iloc[0]  
    
    outliers_list = []
    
    for index, row in df.iterrows():
        reasons = []
        if row.get('DepDelay', 0) > delay_threshold:
            reasons.append('DepDelay > threshold')
        if row.get('ArrDelay', 0) > delay_threshold:
            reasons.append('ArrDelay > threshold')
        if row.get('CarrierDelay', 0) > delay_threshold:
            reasons.append('CarrierDelay > threshold')
        if row.get('WeatherDelay', 0) > delay_threshold:
            reasons.append('WeatherDelay > threshold')
        if row.get('NASDelay', 0) > delay_threshold:
            reasons.append('NASDelay > threshold')
        
        # Apply the 2022 Seattle-specific Alaska Airlines
        if year == 2022:
            if (row.get('Marketing_Airline_Network') == 'AS' or row.get('DOT_ID_Operating_Airline') == 'AS') and \
               (row.get('Origin') == 'SEA' or row.get('Dest') == 'SEA') and \
               (row.get('DepDelay', 0) > delay_threshold or row.get('ArrDelay', 0) > delay_threshold):
                reasons.append('2022 Alaska Airlines Seattle issue')
        
        if reasons:
            outliers_list.append({'ID': index, 'Reason': ', '.join(reasons)})

    outliers_df = pd.DataFrame(outliers_list)
    
    outliers_df = outliers_df.groupby('ID', as_index=False).agg({'Reason': ', '.join}) # Merge ID together
    
    base_name = os.path.basename(file)
    outlier_filename = f"outliers_summary_{base_name}"
    outlier_filepath = os.path.join(folder_path, outlier_filename)
    
    outliers_df.to_csv(outlier_filepath, index=False)
    print(f"Outliers summary saved as {outlier_filename}")


/var/folders/p1/4t2ts09s1pg1p6xt550h6jp80000gn/T/ipykernel_21824/1264380248.py:15: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, encoding='ISO-8859-1')


Outliers summary saved as outliers_summary_20-1.csv
Outliers summary saved as outliers_summary_20-12.csv
Outliers summary saved as outliers_summary_20-11.csv
Outliers summary saved as outliers_summary_22-1.csv
Outliers summary saved as outliers_summary_22-11.csv
Outliers summary saved as outliers_summary_18-11.csv
Outliers summary saved as outliers_summary_22-12.csv
Outliers summary saved as outliers_summary_24-1.csv
Outliers summary saved as outliers_summary_19-1.csv
Outliers summary saved as outliers_summary_18-12.csv
Outliers summary saved as outliers_summary_23-1.csv
Outliers summary saved as outliers_summary_21-1.csv
Outliers summary saved as outliers_summary_18-1.csv


/var/folders/p1/4t2ts09s1pg1p6xt550h6jp80000gn/T/ipykernel_21824/1264380248.py:15: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, encoding='ISO-8859-1')


Outliers summary saved as outliers_summary_21-11.csv
Outliers summary saved as outliers_summary_21-12.csv
Outliers summary saved as outliers_summary_19-12.csv
Outliers summary saved as outliers_summary_23-12.csv
Outliers summary saved as outliers_summary_19-11.csv
Outliers summary saved as outliers_summary_23-11.csv
